# Cadeia de Markov

Entende-se por tempo de passagem, o tempo médio necessário para que as árvores de uma classe diamétrica passe à seguinte.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
continuo = pd.read_csv('https://raw.githubusercontent.com/Gorgens/treelab/master/manejo/inventarioContinuo.csv')

continuoMenor35 = continuo[continuo['dap.2012'] < 35]
continuoCorrMultinivel = pd.concat([continuo, continuoMenor35, continuoMenor35, continuoMenor35, continuoMenor35, continuoMenor35, continuoMenor35, continuoMenor35, continuoMenor35, continuoMenor35])

1. Obter centro de classe
2. Obter incremento anual

In [3]:
intervaloClasse = 10.0
diametroMin = 10.0

classeMax = ((max(continuoCorrMultinivel['dap.2012']) // intervaloClasse) + 1) * intervaloClasse
intervalos = list(np.arange(diametroMin, classeMax, intervaloClasse))
intervalos = [(x - 0.001) for x in intervalos]
centros = [round(x + intervaloClasse/2.0, 0) for x in intervalos[:-1]]
continuoCorrMultinivel = continuoCorrMultinivel[continuoCorrMultinivel['dap.2012'] >= diametroMin]
continuoCorrMultinivel['centroClasse2012'] = pd.cut(continuoCorrMultinivel['dap.2012'], bins=intervalos, labels=centros)

classeMax = ((max(continuoCorrMultinivel['dap.2014']) // intervaloClasse) + 1) * intervaloClasse
intervalos = list(np.arange(diametroMin, classeMax, intervaloClasse))
intervalos = [(x - 0.001) for x in intervalos]
centros = [round(x + intervaloClasse/2.0, 0) for x in intervalos[:-1]]
continuoCorrMultinivel['centroClasse2014'] = pd.cut(continuoCorrMultinivel['dap.2014'], bins=intervalos, labels=centros)

# print(continuo.shape)
continuoCorrMultinivel.sample(10)

,parcela,arv,comum.nome,cientifico.nome,familia.nome,dap.2012,tipo,vert.2012,Luz.2012,Hcom.2012,Htot.2012,dap.2014,UTM.leste,UTM.norte,centroClasse2012,centroClasse2014
1489,T13,73,coquirana,Chrysophyllum sanguinolentum,Sapotaceae,13.0,O,S,2.0,18.01,25.85,13.1,778803.9782,9586260.339,15.0,15.0
973,T09,24,arataciu,Sagotia racemosa,Euphorbiaceae,10.5,O,S,2.0,9.35,13.82,10.5,780492.9391,9585021.382,15.0,15.0
843,T08,1,morta,NaN,NaN,33.0,O,NaN,NaN,NaN,2.08,33.0,779536.2074,9585149.514,35.0,35.0
2533,T22,90,mato,NaN,NaN,14.2,O,S,2.0,27.57,38.06,14.2,779765.8358,9584009.773,15.0,15.0
292,T03,91,louro preto,Ocotea cernua,Lauraceae,38.8,O,C,2.0,21.41,35.78,41.2,779760.8105,9586524.063,35.0,45.0
1746,T15,81,inga branco,Inga thibaudiana,Fabaceae,15.8,O,S,2.0,11.97,17.00,18.5,780117.8787,9585599.770,15.0,15.0
935,T08,169,capueiro,Rinorea guianensis,Violaceae,21.7,O,S,2.0,16.34,22.46,22.2,779535.1725,9585049.108,25.0,25.0
1318,T12,2,jatereu,Lecythis idatimon,Lecythidaceae,10.3,O,S,1.0,2.28,3.94,10.5,778938.8674,9586599.511,15.0,15.0
893,T08,92,matamata,Eschweilera amazonica,Lecythidaceae,23.5,O,S,1.0,13.50,25.64,24.0,779539.6253,9584692.104,25.0,25.0
36,T01,59,matamata,Eschweilera amazonica,Lecythidaceae,22.0,O,S,2.0,11.02,19.12,22.4,778069.3338,9586839.901,25.0,25.0


In [4]:
continuoFiltered = continuoCorrMultinivel[(continuoCorrMultinivel['comum.nome'] != 'morta') &
                                          (continuoCorrMultinivel['comum.nome'] != 'liana') &
                                          (continuoCorrMultinivel['dap.2012'] >= 10.) &
                                          (continuoCorrMultinivel['dap.2014'] >= 10.)]
continuoFiltered.sample(10)

,parcela,arv,comum.nome,cientifico.nome,familia.nome,dap.2012,tipo,vert.2012,Luz.2012,Hcom.2012,Htot.2012,dap.2014,UTM.leste,UTM.norte,centroClasse2012,centroClasse2014
1334,T12,18,matamata branco,Eschweilera coriacea,Lecythidaceae,43.4,O,E,1.0,23.06,33.64,43.7,778947.2496,9586690.275,45.0,45.0
1345,T12,29,arataciu,Sagotia racemosa,Euphorbiaceae,13.8,O,S,3.0,5.26,8.68,13.9,778937.4480,9586726.983,15.0,15.0
1955,T17,49,matamata,Eschweilera amazonica,Lecythidaceae,13.5,O,S,2.0,7.18,9.80,13.8,780798.0402,9585961.521,15.0,15.0
1938,T17,32,capueiro,Rinorea guianensis,Violaceae,27.1,O,S,2.0,12.13,17.46,27.4,780795.5725,9586051.219,25.0,25.0
1573,T14,24,jatereu,Lecythis idatimon,Lecythidaceae,19.7,O,S,2.0,9.84,16.29,20.2,779293.8457,9586758.797,15.0,25.0
1988,T17,83,arataciu,Sagotia racemosa,Euphorbiaceae,10.5,O,S,2.0,9.06,13.62,11.0,780815.1001,9585764.772,15.0,15.0
1251,T11,101,embauba,Cecropia ficifolia,Urticaceae,32.9,O,E,1.0,17.00,26.22,33.6,779595.5253,9584161.319,35.0,35.0
1134,T10,79,breu branco,Protium paniculatum,Burseraceae,17.2,O,S,2.0,8.80,13.42,13.5,778635.9643,9584059.219,15.0,15.0
2217,T19,86,jatereu,Lecythis idatimon,Lecythidaceae,13.8,O,S,2.0,13.49,17.61,13.9,779793.2423,9584650.334,15.0,15.0
1210,T11,27,louro,Ocotea canaliculata,Lauraceae,23.1,O,S,1.0,13.31,19.99,25.1,779380.9120,9584150.639,25.0,25.0


In [60]:
freqTable = continuoFiltered.pivot_table(values='arv', index='centroClasse2014', columns='centroClasse2012', aggfunc='count')
freqTable

centroClasse2012,15.0,25.0,35.0,45.0,55.0,65.0,75.0,85.0,95.0,105.0,115.0,125.0,145.0,155.0,165.0
centroClasse2014,,,,,,,,,,,,,,,
15.0,6660.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25.0,270.0,2410.0,31.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35.0,NaN,90.0,838.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45.0,NaN,NaN,62.0,402.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55.0,NaN,NaN,1.0,24.0,241.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65.0,NaN,NaN,NaN,1.0,17.0,84.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75.0,NaN,NaN,NaN,NaN,1.0,9.0,44.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
85.0,NaN,NaN,NaN,NaN,NaN,1.0,3.0,37.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,16.0,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
freqTable = freqTable.rename_axis(None)
freqTable = freqTable.rename_axis(None, axis='columns')
freqTable.columns=freqTable.columns.astype('float')
freqTable.index=freqTable.index.astype('float')
freqTable.fillna(0, inplace=True)
freqTable

,15.0,25.0,35.0,45.0,55.0,65.0,75.0,85.0,95.0,105.0,115.0,125.0,145.0,155.0,165.0
15.0,6660.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25.0,270.0,2410.0,31.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35.0,0.0,90.0,838.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
45.0,0.0,0.0,62.0,402.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
55.0,0.0,0.0,1.0,24.0,241.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
65.0,0.0,0.0,0.0,1.0,17.0,84.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75.0,0.0,0.0,0.0,0.0,1.0,9.0,44.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
85.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,37.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
95.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0
105.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,0.0,0.0,0.0,0.0,0.0


In [64]:
for c in freqTable.columns:
  for l in freqTable.index:
    if l < c:
      freqTable.loc[l, c] = 0
    elif l > c + intervaloClasse:
      freqTable.loc[l, c] = 0

freqTable   

,15.0,25.0,35.0,45.0,55.0,65.0,75.0,85.0,95.0,105.0,115.0,125.0,145.0,155.0,165.0
15.0,6660.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25.0,270.0,2410.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35.0,0.0,90.0,838.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
45.0,0.0,0.0,62.0,402.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
55.0,0.0,0.0,0.0,24.0,241.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
65.0,0.0,0.0,0.0,0.0,17.0,84.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75.0,0.0,0.0,0.0,0.0,0.0,9.0,44.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
85.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,37.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
95.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0
105.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,0.0,0.0,0.0,0.0,0.0


In [73]:
corte = 85. // intervaloClasse

freqTable = freqTable.iloc[0:int(corte+1),0:int(corte)]
freqTable

,15.0,25.0,35.0,45.0,55.0,65.0,75.0,85.0
15.0,6660.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25.0,270.0,2410.0,0.0,0.0,0.0,0.0,0.0,0.0
35.0,0.0,90.0,838.0,0.0,0.0,0.0,0.0,0.0
45.0,0.0,0.0,62.0,402.0,0.0,0.0,0.0,0.0
55.0,0.0,0.0,0.0,24.0,241.0,0.0,0.0,0.0
65.0,0.0,0.0,0.0,0.0,17.0,84.0,0.0,0.0
75.0,0.0,0.0,0.0,0.0,0.0,9.0,44.0,0.0
85.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,37.0
95.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0


In [74]:
freqTable.div(freqTable.sum(axis=0), axis=1)

,15.0,25.0,35.0,45.0,55.0,65.0,75.0,85.0
15.0,0.961039,0.000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
25.0,0.038961,0.964,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
35.0,0.000000,0.036,0.931111,0.000000,0.000000,0.000000,0.00000,0.000000
45.0,0.000000,0.000,0.068889,0.943662,0.000000,0.000000,0.00000,0.000000
55.0,0.000000,0.000,0.000000,0.056338,0.934109,0.000000,0.00000,0.000000
65.0,0.000000,0.000,0.000000,0.000000,0.065891,0.903226,0.00000,0.000000
75.0,0.000000,0.000,0.000000,0.000000,0.000000,0.096774,0.93617,0.000000
85.0,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.06383,0.902439
95.0,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00000,0.097561
